#  Chains

Chains allow us to link the output of one LLM call as input of another LLM call.

Chain topics explored:

1) LLMChain
2) SimpleSequentialChain
3) SequentialChain
4) LLMRouterChain
5) TransformChain
6) OpenAI Function Calling
7) MathChain
8) AdditionalChains

In [3]:
import os
openai_api_key = os.getenv(key="OPENAI_API_KEY")

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

Chains have a basic building block called LLMChain object. We can think of an LLMChain object as simple LLM call that will have an input and an output.

In [7]:
human_message_prompt = HumanMessagePromptTemplate.from_template("Make up a funny company name for a company that produces {product}")
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])

In [8]:
chat = ChatOpenAI()

In [9]:
from langchain.chains import LLMChain
chain = LLMChain(llm=chat, prompt=chat_prompt_template)

In [11]:
print(chain.run(product="Soap"))

Sudsy McSuds Soap Co.


## SimpleSequentialCHain

The flow of a SimpleSequentialChain is something like:

input --> LLMChain --> LLMChain --> LLMChain --> Output

In [21]:
from langchain.chains import SimpleSequentialChain

In [26]:
llm = ChatOpenAI()

In [28]:
# 2 LLMChain blocks: Topic for a blog post --> [[ Outline --> Create blog post from outline ]] --> Blog post text

#First block that will output the outline
template = "Give me a simple bullet point outline for a blog post on {topic}"
first_prompt = ChatPromptTemplate.from_template(template)
chain_one = LLMChain(llm = llm, prompt = first_prompt)

In [29]:
#Second block
template_two = "Write a blog post using this outline: {outline}"
second_prompt = ChatPromptTemplate.from_template(template_two)
chain_two = LLMChain(llm = llm, prompt = second_prompt)

In [30]:
full_chain = SimpleSequentialChain(chains = [chain_one, chain_two], verbose = True)

In [ ]:
full_chain

In [33]:
result = full_chain.run("Albert Einstein")



> Entering new SimpleSequentialChain chain...
- Introduction to Albert Einstein and his contributions to science
- Early life and education of Albert Einstein
- The theory of relativity and its impact on physics
- Other significant achievements and awards received by Einstein
- Legacy of Albert Einstein and his influence on modern science
- Conclusion and reflection on the impact of Einstein's work
Albert Einstein is undoubtedly one of the most well-known and influential scientists in history. His groundbreaking work in the field of physics revolutionized our understanding of the universe and laid the foundation for many of the scientific advancements we enjoy today. In this blog post, we will explore the life and contributions of Albert Einstein, from his early beginnings to his lasting legacy in the world of science.

Born in 1879 in Ulm, Germany, Albert Einstein showed an early aptitude for math and science. After moving to Switzerland with his family, he continued his education a

In [34]:
print(result)

Albert Einstein is undoubtedly one of the most well-known and influential scientists in history. His groundbreaking work in the field of physics revolutionized our understanding of the universe and laid the foundation for many of the scientific advancements we enjoy today. In this blog post, we will explore the life and contributions of Albert Einstein, from his early beginnings to his lasting legacy in the world of science.

Born in 1879 in Ulm, Germany, Albert Einstein showed an early aptitude for math and science. After moving to Switzerland with his family, he continued his education and eventually went on to study physics at the Swiss Federal Institute of Technology in Zurich. It was during this time that Einstein began to develop his revolutionary theories on the nature of space and time.

One of Einstein's most famous and groundbreaking theories is the theory of relativity, which he first published in 1905. This theory fundamentally changed our understanding of how the universe 

In [35]:
type(result)

str

## SequentialChain

Sequential Chains are similar to simple sequential chains but allow us to have access to the outputs from the internal LLMs

In [1]:
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain

In [6]:
llm = ChatOpenAI()

In [7]:
# 3 Chains
# Employee performance review input text
# review_text --> LLMChain --> summary
# Summary --> LLMChain --> Weaknesses
# Weaknesses --> LLMChain --> Improvement Plan

In [8]:
template1 = "Give a summary of this employee's performance review:\n{review}"
prompt1 = ChatPromptTemplate.from_template(template1)
chain1 = LLMChain(llm=llm, prompt=prompt1, output_key="review_summary")  # specify output in order to be used in the next chain

In [10]:
template2 = "Identify key employee weaknesses from this performance review summary:\n{review_summary}"  # here we include the output of the previous
prompt2 = ChatPromptTemplate.from_template(template2)
chain2 = LLMChain(llm=llm, prompt=prompt2, output_key="weaknesses")

In [11]:
template3 = "Create a personalised improvement plan to help address and fix these weaknesses:\n{weaknesses}"  # needs to match output key
prompt3 = ChatPromptTemplate.from_template(template3)
chain3 = LLMChain(llm=llm, prompt=prompt3, output_key="final_plan")

In [12]:
employee_review = '''
Employee Information:
Name: Joe Schmo
Position: Software Engineer
Date of Review: July 14, 2023

Strengths:
Joe is a highly skilled software engineer with a deep understanding of programming languages, algorithms, and software development best practices. His technical expertise shines through in his ability to efficiently solve complex problems and deliver high-quality code.

One of Joe's greatest strengths is his collaborative nature. He actively engages with cross-functional teams, contributing valuable insights and seeking input from others. His open-mindedness and willingness to learn from colleagues make him a true team player.

Joe consistently demonstrates initiative and self-motivation. He takes the lead in seeking out new projects and challenges, and his proactive attitude has led to significant improvements in existing processes and systems. His dedication to self-improvement and growth is commendable.

Another notable strength is Joe's adaptability. He has shown great flexibility in handling changing project requirements and learning new technologies. This adaptability allows him to seamlessly transition between different projects and tasks, making him a valuable asset to the team.

Joe's problem-solving skills are exceptional. He approaches issues with a logical mindset and consistently finds effective solutions, often thinking outside the box. His ability to break down complex problems into manageable parts is key to his success in resolving issues efficiently.

Weaknesses:
While Joe possesses numerous strengths, there are a few areas where he could benefit from improvement. One such area is time management. Occasionally, Joe struggles with effectively managing his time, resulting in missed deadlines or the need for additional support to complete tasks on time. Developing better prioritization and time management techniques would greatly enhance his efficiency.

Another area for improvement is Joe's written communication skills. While he communicates well verbally, there have been instances where his written documentation lacked clarity, leading to confusion among team members. Focusing on enhancing his written communication abilities will help him effectively convey ideas and instructions.

Additionally, Joe tends to take on too many responsibilities and hesitates to delegate tasks to others. This can result in an excessive workload and potential burnout. Encouraging him to delegate tasks appropriately will not only alleviate his own workload but also foster a more balanced and productive team environment.
'''

In [13]:
full_chain = SequentialChain(
    chains=[chain1, chain2, chain3],
    input_variables=["review"],  # include the input of first chain
    output_variables=["review_summary", "weaknesses", "final_plan"],  # it's a good practice to include all of the variables
    verbose=True
)

In [14]:
result = full_chain(employee_review)



> Entering new SequentialChain chain...

> Finished chain.


In [16]:
type(result)

dict

In [18]:
result.keys()

dict_keys(['review', 'review_summary', 'weaknesses', 'final_plan'])

In [22]:
print(result["review"])


Employee Information:
Name: Joe Schmo
Position: Software Engineer
Date of Review: July 14, 2023

Strengths:
Joe is a highly skilled software engineer with a deep understanding of programming languages, algorithms, and software development best practices. His technical expertise shines through in his ability to efficiently solve complex problems and deliver high-quality code.

One of Joe's greatest strengths is his collaborative nature. He actively engages with cross-functional teams, contributing valuable insights and seeking input from others. His open-mindedness and willingness to learn from colleagues make him a true team player.

Joe consistently demonstrates initiative and self-motivation. He takes the lead in seeking out new projects and challenges, and his proactive attitude has led to significant improvements in existing processes and systems. His dedication to self-improvement and growth is commendable.

Another notable strength is Joe's adaptability. He has shown great flexi

In [23]:
print(result["review_summary"])

Overall, Joe Schmo is a highly skilled and collaborative software engineer with exceptional problem-solving abilities and adaptability. He demonstrates initiative, self-motivation, and a willingness to learn from others. However, he could benefit from improving his time management skills, written communication, and delegation of tasks. By addressing these areas for improvement, Joe can further enhance his efficiency and contribute even more effectively to the team.


In [24]:
print(result["weaknesses"])

1. Time management skills
2. Written communication
3. Delegation of tasks


## LLMRouterChain

Takes an input and redirects it to the most appropriate LLMChain sequence
The router accepts multiple potential destination LLMChains and then via a specialized prompt, the router will reat the initial input then output a spefic dictionary that matches up to one of the potential destination chains to continue processing.

In [27]:
# person ask question about football
# "What is the main objective of a football game?"
# "Who are three of the best football players of all time and what made them so technically good?"
# Questions that can be really simple or difficult
# Input --> Router --> LLM decides the correct chain --> Chain --> Output

In [28]:
from langchain.chains import RouterChain

In [29]:
begginer_template = """You are an expert in football who is really focused on beginners and explaining complex concepts in simple to understand terms.
You assume no prior knowledge. Here is your question:\n{input}"""

In [30]:
expert_template = """You are an expert in football who knows a lot about football and the players and explains topics to advanced audience members. 
You can assume anyone you answer loves football. Here is your question:\n{input}"""

In [31]:
# Route prompt information
# [{Name, Description, Template}]

prompt_infos = [
    {
        "name": "Beginner Football",
        "description": "Answers basic football questions",  # Important that this matches the template phrasing.
        "template": begginer_template,
    },
    {
        "name": "Expert Football",
        "description": "Answers advanced football questions",  # Important that this matches the template phrasing.
        "template": expert_template,
    }
]

In [32]:
llm = ChatOpenAI()

destination_chains = {}

for p_info in prompt_infos:
    name = p_info["name"]
    template = p_info["template"]

    prompt = ChatPromptTemplate.from_template(template=template)

    chain = LLMChain(llm=llm, prompt=prompt)

    destination_chains[name] = chain

In [33]:
destination_chains.keys()

dict_keys(['Beginner Football', 'Expert Football'])

In [34]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [35]:
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [36]:
# This is just a set of instructions
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [37]:
# Edit the MULTI PROMPT ROUTER TEMPLATE
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations

['Beginner Football: Answers basic football questions',
 'Expert Football: Answers advanced football questions']

In [38]:
destination_str = "\n".join(destinations)  # we should follow this format
destination_str

'Beginner Football: Answers basic football questions\nExpert Football: Answers advanced football questions'

In [39]:
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
# Destinations are now filled
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destination_str
)
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
Beginner Football: Answers basic football questions
Expert Football: Answers advanced football questi

In [40]:
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)
from langchain.chains.router import MultiPromptChain
router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt)
chain = MultiPromptChain(
    router_chain=router_chain, 
    destination_chains=destination_chains, 
    default_chain=default_chain,
    verbose=True
)

In [41]:
result = chain.run("What is the objective of a football game?")



> Entering new MultiPromptChain chain...
Beginner Football: {'input': 'What is the objective of a football game?'}
> Finished chain.


In [42]:
print(result)

The objective of a football game is for each team to score more points than the other team by moving the ball into the opposing team's end zone. Each team has a set amount of time, known as quarters, to try and score as many points as possible. Points can be scored by either carrying the ball into the end zone for a touchdown, kicking the ball through the goalposts for a field goal, or tackling an opposing player in their own end zone for a safety. The team with the most points at the end of the game is declared the winner.


In [43]:
result = chain.run("Who are three of the best football players of all time and what made them so technically good?")
print(result)



> Entering new MultiPromptChain chain...
None: {'input': 'Who are three of the best football players of all time and what made them so technically good?'}
> Finished chain.
1. Lionel Messi - Messi is widely regarded as one of the greatest footballers of all time due to his incredible dribbling ability, vision, and precise finishing. His close ball control and quick acceleration make him almost impossible to defend against, and his ability to create scoring opportunities for himself and his teammates is unparalleled.

2. Cristiano Ronaldo - Ronaldo is another player often considered one of the best of all time due to his exceptional physical attributes and goal-scoring ability. His speed, strength, and aerial prowess make him a constant threat in front of goal, and his work ethic and dedication to training have allowed him to maintain a high level of performance throughout his career.

3. Diego Maradona - Maradona is a legendary player known for his incredible skill on the ball, agili

## Transform Chain

In [44]:
yelp_review = open("yelp_review.txt").read()

In [45]:
print(yelp_review)

TITLE: AN ABSOLUTE DELIGHT! A CULINARY HAVEN!

REVIEW:
OH MY GOODNESS, WHERE DO I BEGIN? THIS RESTAURANT IS ABSOLUTELY PHENOMENAL! I WENT THERE LAST NIGHT WITH MY FRIENDS, AND WE WERE BLOWN AWAY BY THE EXPERIENCE!

FIRST OF ALL, THE AMBIANCE IS OUT OF THIS WORLD! THE MOMENT YOU STEP INSIDE, YOU'RE GREETED WITH A WARM AND INVITING ATMOSPHERE. THE DECOR IS STUNNING, AND IT IMMEDIATELY SETS THE TONE FOR AN UNFORGETTABLE DINING EXPERIENCE.

NOW, LET'S TALK ABOUT THE FOOD! WOW, JUST WOW! THE MENU IS A PARADISE FOR FOOD LOVERS. EVERY DISH WE ORDERED WAS A MASTERPIECE. THE FLAVORS WERE BOLD, VIBRANT, AND EXPLODED IN OUR MOUTHS. FROM STARTERS TO DESSERTS, EVERY BITE WAS PURE BLISS!

THEIR SEAFOOD PLATTER IS A MUST-TRY! THE FRESHNESS OF THE SEAFOOD IS UNMATCHED, AND THE PRESENTATION IS SIMPLY STUNNING. I HAVE NEVER TASTED SUCH DELICIOUS AND PERFECTLY COOKED SEAFOOD IN MY LIFE. IT'S A SEAFOOD LOVER'S DREAM COME TRUE!

THE SERVICE WAS EXEMPLARY. THE STAFF WAS ATTENTIVE, FRIENDLY, AND EXTREMELY KN

In [46]:
# We want the text to be in lowercase and only the review section
print(yelp_review.split("REVIEW:")[-1].lower())


oh my goodness, where do i begin? this restaurant is absolutely phenomenal! i went there last night with my friends, and we were blown away by the experience!

first of all, the ambiance is out of this world! the moment you step inside, you're greeted with a warm and inviting atmosphere. the decor is stunning, and it immediately sets the tone for an unforgettable dining experience.

now, let's talk about the food! wow, just wow! the menu is a paradise for food lovers. every dish we ordered was a masterpiece. the flavors were bold, vibrant, and exploded in our mouths. from starters to desserts, every bite was pure bliss!

their seafood platter is a must-try! the freshness of the seafood is unmatched, and the presentation is simply stunning. i have never tasted such delicious and perfectly cooked seafood in my life. it's a seafood lover's dream come true!

the service was exemplary. the staff was attentive, friendly, and extremely knowledgeable about the menu. they went above and beyond

In [47]:
# Now we want to do internal operations like Input --> custom python transformation --> LLMChain

In [48]:
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [49]:
def transformer_fun(inputs: dict) -> dict:
    text = inputs["text"]
    only_review_text = text.split("REVIEW")[-1]
    lower_case_text = only_review_text.lower()
    return {"output": lower_case_text}

In [50]:
transform_chain = TransformChain(
    input_variables=["text"],
    output_variables=["output"],
    transform=transformer_fun
)

In [51]:
template = "Create a one sentence summary of this review:\n{review}"

In [52]:
llm = ChatOpenAI()
prompt = ChatPromptTemplate.from_template(template)
summary_chain = LLMChain(llm=llm, prompt=prompt, output_key="review_summary")

In [53]:
sequential_chain = SimpleSequentialChain(
    chains=[transform_chain, summary_chain],
    verbose=True
)

In [54]:
result = sequential_chain(yelp_review)



> Entering new SimpleSequentialChain chain...
:
oh my goodness, where do i begin? this restaurant is absolutely phenomenal! i went there last night with my friends, and we were blown away by the experience!

first of all, the ambiance is out of this world! the moment you step inside, you're greeted with a warm and inviting atmosphere. the decor is stunning, and it immediately sets the tone for an unforgettable dining experience.

now, let's talk about the food! wow, just wow! the menu is a paradise for food lovers. every dish we ordered was a masterpiece. the flavors were bold, vibrant, and exploded in our mouths. from starters to desserts, every bite was pure bliss!

their seafood platter is a must-try! the freshness of the seafood is unmatched, and the presentation is simply stunning. i have never tasted such delicious and perfectly cooked seafood in my life. it's a seafood lover's dream come true!

the service was exemplary. the staff was attentive, friendly, and extremely knowled

In [55]:
print(result["output"])

This restaurant offers a phenomenal dining experience with stunning ambiance, delicious food, and exemplary service that is sure to leave you wanting more.


## Math Chain

In [56]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage

In [57]:
model = ChatOpenAI()
result = model([HumanMessage(content="What is 2 + 2")])

In [59]:
print(result.content)

2 + 2 equals 4.


In [60]:
17**11

34271896307633

In [62]:
result = model([HumanMessage(content="What is 17 raised to the power of 11")])
print(result.content)
#Hallucinated the result

17^11 is equal to 1,420,475,085,111


In [65]:
from langchain import LLMMathChain
llm_math_model = LLMMathChain.from_llm(model)

In [66]:
llm_math_model("What is 17 raised to the power of 11")

{'question': 'What is 17 raised to the power of 11',
 'answer': 'Answer: 34271896307633'}

## Additional chains - QA Documents

There are some additional pre-built chains, let's explore examples for two of the most commonly used chains for documents QA. 

In [67]:
# Vector store
# QA on that vector store
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [79]:
embedding_function = OpenAIEmbeddings()
db_connection = Chroma(persist_directory="US_Constitution", embedding_function=embedding_function)

In [80]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain # very similar, more metadata
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)

In [81]:
chain = load_qa_chain(
    llm=llm,
    chain_type="stuff"  # stuffing in some context from vector store
)

In [82]:
question = "What is the 15th amendment?"

In [83]:
docs = db_connection.similarity_search(question)  # Document that will be context
chain = load_qa_with_sources_chain(llm, chain_type="stuff")
chain.run(
    input_documents=docs,
    question=question
)

'The 15th Amendment states that the right of citizens of the United States to vote shall not be denied or abridged by the United States or by any State on account of race, color, or previous condition of servitude.\nSOURCES: some_data/US_Constitution.txt'